In [6]:
import os
import pickle
import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy as sp
import scipy.linalg as linalg
import sklearn as skl
import pandas as pd
import json
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
# Descargo y preparo los dataset para el autoencoder
transform = transforms.Compose([transforms.ToTensor()])

train_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = True,  transform = transform)
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)

class CustomDataSet(Dataset):
    def __init__(self,dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self,i):
        image, label = self.dataset[i]
        input = image
        output = image
        return input,output

train_set = CustomDataSet(train_set_orig)
valid_set = CustomDataSet(valid_set_orig)

In [9]:
# Calcula la dimensión de la salida de una convolución 2D
# Conv2D de PyTorch es así: torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)    

def convt2d_z_out(z_in, kernel_size,output_padding, stride = 1, padding = 0, dilation = 1):
    z_out=(z_in-1)*stride-2*padding+dilation*(kernel_size-1)+output_padding
    return z_out

In [10]:
class Autoencoder_Conv(nn.Module):
    def __init__(self, n=28*28,p=0.2):
        super().__init__()             
        self.n = n
        self.p = p

        #Encoder
        self.encoder = nn.Sequential(
            
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3), #(1, 28, 28) -> (16, 26, 26)
            nn.ReLU(),
            nn.Dropout(self.p),
            nn.MaxPool2d(kernel_size=2,stride=2), # (16, 26, 26) -> (16, 13, 13)
            
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3), # (16, 13, 13) a (32, 11, 11)
            nn.ReLU(),
            nn.Dropout(self.p),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32, 11, 11) -> (32, 5, 5)

            
            nn.Flatten(),
            nn.Linear(32*5*5,self.n), #(32, 5, 5) -> 32*5*5
            nn.ReLU(),
            nn.Dropout(self.p) # 32*5*5 -> n
        )

        # Decoder
        self.decoder = nn.Sequential(

            nn.Linear(in_features=self.n,out_features=32*5*5), # n -> 32*5*5
            nn.ReLU(),
            nn.Dropout(self.p),
            nn.Unflatten(1,(32,5,5)), # 32*5*5 -> (32, 5, 5)

            nn.ConvTranspose2d(in_channels=32,out_channels=16,kernel_size=5,stride=2,padding=2,output_padding=0,dilation=2), # (32, 5, 5) -> (16, 13, 13)
            nn.ReLU(),
            nn.Dropout(self.p),

            nn.ConvTranspose2d(in_channels=16,out_channels=1,kernel_size=6,stride=2,padding=1,output_padding=0,dilation=1), # (16, 13, 13) -> (1, 28, 28)
            nn.Sigmoid()
        )

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
class AutoencoderConv2(nn.Module):
    def __init__(self, n=128, p=0.2):
        super().__init__()
        self.n = n
        self.p = p

        # Encoder
        self.encoder = nn.Sequential(
            # Capa 1: de (1,28,28) a (32,28,28)
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (32,28,28) -> (32,14,14)
            nn.Dropout(p),
            
            # Capa 2: de (32,14,14) a (64,14,14)
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (64,14,14) -> (64,7,7)
            nn.Dropout(p),
            
            nn.Flatten(),  # (64,7,7) -> 3136
            nn.Linear(3136, n),
            nn.ReLU(),
            nn.Dropout(p)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(n, 3136),
            nn.ReLU(),
            nn.Dropout(p),
            nn.Unflatten(1, (64, 7, 7)),
            
            # Capa 1: de (64,7,7) a (32,14,14)
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, 
                               padding=1, output_padding=1),
            nn.ReLU(),
            nn.Dropout(p),
            
            # Capa 2: de (32,14,14) a (1,28,28)
            nn.ConvTranspose2d(in_channels=32, out_channels=1, kernel_size=3, stride=2, 
                               padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [12]:
def batch(x):
  return x.to(device).unsqueeze(0) # (28,28) --> (1,28,28)

def unbatch(x):
  return x.squeeze().detach().cpu().numpy() # (28,28) --> (1,28,28)

In [13]:
# Verifico que el modelo sea coherente en input size y output size
model = Autoencoder_Conv()
image,_  = train_set[1]
image = image.to(device)
pred = model(batch(image))
print(image.size())
print(unbatch(image).shape)
print(pred.size())
print(unbatch(pred).shape)

torch.Size([1, 28, 28])
(28, 28)
torch.Size([1, 1, 28, 28])
(28, 28)


In [16]:
# Función de entrenamiento

def train_loop(dataLoader,model,loss_fn,optimizer,verbose=True):

  model.train()

  num_samples = len(dataLoader.dataset)
  num_batches = len(dataLoader)
  sum_batch_avg_loss = 0
  num_processed_samples = 0
  
  
  for batch, (X,y) in enumerate(dataLoader):

    batch_size = len(X)
    num_processed_samples += batch_size

    # Calculo la prediccion del modelo y el error
    pred = model(X)
    loss = loss_fn(pred,y)

    # Anulo el gradiente, hago backpropagation para calcular los gradientes de nuevo, y actualizo los pesos
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Registro el progreso
    batch_avg_loss = loss.item()
    sum_batch_avg_loss += batch_avg_loss
    if batch % (num_batches/10) == 0 and verbose:
      print(f"Número de batch: {batch:>5d}, Pérdida promedio del batch: {batch_avg_loss:>7f}, Muestras procesadas:[{num_processed_samples:>5d}/{num_samples:>5d}]")
      
  avg_loss = sum_batch_avg_loss/num_batches
  return avg_loss


# Funcion de validacion

def eval_loop(dataLoader,model,loss_fn):

  model.eval()

  num_samples = len(dataLoader.dataset)
  num_batches = len(dataLoader)
  sum_batch_avg_loss = 0
  num_processed_samples = 0

  with torch.inference_mode():

    for X,y in dataLoader:

      batch_size = len(X)
      num_processed_samples += batch_size

      # Calculo la prediccion del modelo y el error
      pred = model(X)
      loss = loss_fn(pred,y)

      # Registro el progreso
      batch_avg_loss = loss.item()
      sum_batch_avg_loss += batch_avg_loss
  
  avg_loss = sum_batch_avg_loss/num_batches
  return avg_loss

In [ ]:
# Entrenamiento Modelo 1: n = 256; p = 0.2; ADAM; LR = 1e-3

n = 256
p = 0.2
autoencoder_conv_1 = Autoencoder_Conv(n=n, p=p)

model1 = autoencoder_conv_1
learning_rate = 1e-3

optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 100
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_1 = []
list_avg_train_loss_1 = []
list_avg_valid_loss_1 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model1, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model1, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model1, loss_fn)
  list_avg_train_loss_incorrecta_1.append(avg_train_loss_incorrecta)
  list_avg_train_loss_1.append(avg_train_loss)
  list_avg_valid_loss_1.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results1 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_1,
    'ECM sobre conjunto de entrenamiento n=256 p=0.2': list_avg_train_loss_1,
    'ECM sobre conjunto de validación n=256 p=0.2': list_avg_valid_loss_1,
    'model1_state_dict': model1.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Entrenamiento Modelo 2: n = 128; p = 0.2; ADAM; LR = 1e-3

n = 128
p = 0.2
autoencoder_conv_2 = Autoencoder_Conv(n=n, p=p)

model2 = autoencoder_conv_2
learning_rate = 1e-3

optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 100
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_2 = []
list_avg_train_loss_2 = []
list_avg_valid_loss_2 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model2, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model2, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model2, loss_fn)
  list_avg_train_loss_incorrecta_2.append(avg_train_loss_incorrecta)
  list_avg_train_loss_2.append(avg_train_loss)
  list_avg_valid_loss_2.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results2 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_2,
    'ECM sobre conjunto de entrenamiento n=128 p=0.2': list_avg_train_loss_2,
    'ECM sobre conjunto de validación n=128 p=0.2': list_avg_valid_loss_2,
    'model2_state_dict': model2.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Entrenamiento Modelo 3: n = 64; p = 0.2; ADAM; LR = 1e-3

n = 64
p = 0.2
autoencoder_conv_3 = Autoencoder_Conv(n=n, p=p)

model3 = autoencoder_conv_3
learning_rate = 1e-3

optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 100
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_3 = []
list_avg_train_loss_3 = []
list_avg_valid_loss_3 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model3, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model3, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model3, loss_fn)
  list_avg_train_loss_incorrecta_3.append(avg_train_loss_incorrecta)
  list_avg_train_loss_3.append(avg_train_loss)
  list_avg_valid_loss_3.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results3 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_3,
    'ECM sobre conjunto de entrenamiento n=64 p=0.2': list_avg_train_loss_3,
    'ECM sobre conjunto de validación n=64 p=0.2': list_avg_valid_loss_3,
    'model3_state_dict': model3.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# A partir de Modelo 1, que es el que mejor performea, se busca el mejor modelo variando la probabilidad de dropout

# Entrenamiento Modelo 4: n = 256; p = 0.1; ADAM; LR = 1e-3

n = 256
p = 0.1
autoencoder_conv_4 = Autoencoder_Conv(n=n, p=p)

model4 = autoencoder_conv_4
learning_rate = 1e-3

optimizer = torch.optim.Adam(model4.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 100
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_4 = []
list_avg_train_loss_4 = []
list_avg_valid_loss_4 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model4, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model4, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model4, loss_fn)
  list_avg_train_loss_incorrecta_4.append(avg_train_loss_incorrecta)
  list_avg_train_loss_4.append(avg_train_loss)
  list_avg_valid_loss_4.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results4 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_4,
    'ECM sobre conjunto de entrenamiento n=256 p=0.1': list_avg_train_loss_4,
    'ECM sobre conjunto de validación n=256 p=0.1': list_avg_valid_loss_4,
    'model4_state_dict': model4.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Entrenamiento Modelo 5: n = 256; p = 0.1; ADAM; LR = 1e-2

n = 256
p = 0.1
autoencoder_conv_5 = Autoencoder_Conv(n=n, p=p)

model5 = autoencoder_conv_5
learning_rate = 1e-2

optimizer = torch.optim.Adam(model5.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 100
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_5 = []
list_avg_train_loss_5 = []
list_avg_valid_loss_5 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model5, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model5, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model5, loss_fn)
  list_avg_train_loss_incorrecta_5.append(avg_train_loss_incorrecta)
  list_avg_train_loss_5.append(avg_train_loss)
  list_avg_valid_loss_5.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results5 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_5,
    'ECM sobre conjunto de entrenamiento n=256 p=0.1 lr=1e-2': list_avg_train_loss_5,
    'ECM sobre conjunto de validación n=256 p=0.1 lr=1e-2': list_avg_valid_loss_5,
    'model5_state_dict': model5.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# A partir de Modelo 4, que es el que mejor performea, se busca el mejor modelo variando el tamaño del los batches

# Entrenamiento Modelo 6: n = 256; p = 0.1; ADAM; LR = 1e-3; batch_size = 200

n = 256
p = 0.1
autoencoder_conv_6 = Autoencoder_Conv(n=n, p=p)

model6 = autoencoder_conv_6
learning_rate = 1e-3

optimizer = torch.optim.Adam(model6.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 200
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_6 = []
list_avg_train_loss_6 = []
list_avg_valid_loss_6 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model6, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model6, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model6, loss_fn)
  list_avg_train_loss_incorrecta_6.append(avg_train_loss_incorrecta)
  list_avg_train_loss_6.append(avg_train_loss)
  list_avg_valid_loss_6.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results6 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_6,
    'ECM sobre conjunto de entrenamiento n=256 p=0.1 bs=200': list_avg_train_loss_6,
    'ECM sobre conjunto de validación n=256 p=0.1 bs=200': list_avg_valid_loss_6,
    'model6_state_dict': model6.state_dict(),  # Guarda los pesos del modelo
}


In [ ]:
# Entrenamiento Modelo 7: n = 256; p = 0.1; ADAM; LR = 1e-3; batch_size = 50

n = 256
p = 0.1
autoencoder_conv_7 = Autoencoder_Conv(n=n, p=p)

model7 = autoencoder_conv_7
learning_rate = 1e-3

optimizer = torch.optim.Adam(model7.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 50
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_7 = []
list_avg_train_loss_7 = []
list_avg_valid_loss_7 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model7, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model7, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model7, loss_fn)
  list_avg_train_loss_incorrecta_7.append(avg_train_loss_incorrecta)
  list_avg_train_loss_7.append(avg_train_loss)
  list_avg_valid_loss_7.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results7 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_7,
    'ECM sobre conjunto de entrenamiento n=256 p=0.1 bs=50': list_avg_train_loss_7,
    'ECM sobre conjunto de validación n=256 p=0.1 bs=50': list_avg_valid_loss_7,
    'model7_state_dict': model7.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Entrenamiento Modelo 8: n = 256; p = 0.1; SGD; LR = 1e-3; batch_size = 50

n = 256
p = 0.1
autoencoder_conv_8 = Autoencoder_Conv(n=n, p=p)

model8 = autoencoder_conv_8
learning_rate = 1e-3

optimizer = torch.optim.SGD(model8.parameters(), lr=learning_rate)

batch_size = 50
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_8 = []
list_avg_train_loss_8 = []
list_avg_valid_loss_8 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model8, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model8, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model8, loss_fn)
  list_avg_train_loss_incorrecta_8.append(avg_train_loss_incorrecta)
  list_avg_train_loss_8.append(avg_train_loss)
  list_avg_valid_loss_8.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results8 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_8,
    'ECM sobre conjunto de entrenamiento n=256 p=0.1 SGD': list_avg_train_loss_8,
    'ECM sobre conjunto de validación n=256 p=0.1 SGD': list_avg_valid_loss_8,
    'model8_state_dict': model8.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Entrenamiento Modelo 9: n = 256; p = 0.1; convolución diferente; ADAM; LR = 1e-3; batch_size = 50; 

n = 256
p = 0.1

model_conv2 = AutoencoderConv2(n=n, p=p).to(device)

learning_rate = 1e-3

optimizer = torch.optim.Adam(model_conv2.parameters(), lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)

batch_size = 50
train_loader = DataLoader (train_set,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader (valid_set,batch_size=batch_size,shuffle=True)

loss_fn = nn.MSELoss()

num_epochs = 20

list_avg_train_loss_incorrecta_conv2 = []
list_avg_train_loss_conv2 = []
list_avg_valid_loss_conv2 = []

patience = 10  # Número de épocas sin mejora antes de detenerse
min_delta = 0.001  # Mínima mejora en la pérdida de validación para considerarla una mejora

best_valid_loss = float('inf')  # Inicializamos con un valor infinito
epochs_without_improvement = 0

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  avg_train_loss_incorrecta = train_loop(train_loader, model_conv2, loss_fn, optimizer)
  avg_train_loss = eval_loop(train_loader, model_conv2, loss_fn)
  avg_valid_loss = eval_loop(valid_loader, model_conv2, loss_fn)
  list_avg_train_loss_incorrecta_conv2.append(avg_train_loss_incorrecta)
  list_avg_train_loss_conv2.append(avg_train_loss)
  list_avg_valid_loss_conv2.append(avg_valid_loss)
  print("Pérdida promedio incorrecta de entrenamiento:",avg_train_loss_incorrecta)
  print("Pérdida promedio de entrenamiento:",avg_train_loss)
  print("Pérdida promedio de validación:",avg_valid_loss)

  # Early stopping check 
  if avg_valid_loss < best_valid_loss - min_delta:
        best_valid_loss = avg_valid_loss
        epochs_without_improvement = 0
  else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping en la época {epoch + 1}")
            break  # Salimos del bucle si no hay mejora

print("Done!")

results_conv2 = {
    'ECM incorrectos sobre conjunto de validación': list_avg_train_loss_incorrecta_conv2,
    'ECM sobre conjunto de entrenamiento conv2': list_avg_train_loss_conv2,
    'ECM sobre conjunto de validación con2': list_avg_valid_loss_conv2,
    'model_state_dict': model_conv2.state_dict(),  # Guarda los pesos del modelo
}

In [ ]:
# Ruta al archivo donde se guardaron los resultados
filename1 = 'linealn256p02.pth'
filename2 = 'linealn128p02.pth'
filename3 = 'linealn64p02.pth'
filename4 = 'linealn256p01.pth'
filename5 = 'linealn256p01lr1e-2.pth'
filename6 = 'linealn256p01bs200.pth'
filename7 = 'linealn256p01bs50.pth'
filename8 = 'linealn256p01SGD.pth'
filename9 = 'conv2.pth'



filepath1 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename1)
filepath2 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename2)
filepath3 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename3)
filepath4 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename4)
filepath5 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename5)
filepath6 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename6)
filepath7 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename7)
filepath8 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename8)
filepath9 = os.path.join('/Users/augusto/Documents/Python/Redes Neuronales', filename9)


#torch.save(results1, filepath1)
#torch.save(results2, filepath2)
#torch.save(results3, filepath3)
#torch.save(results4, filepath4)
#torch.save(results5, filepath5)
#torch.save(results6, filepath6)
#torch.save(results7, filepath7)
#torch.save(results8, filepath8)
#torch.save(results_conv2, filepath9)


In [ ]:
# Verificación de que Autoencoder efectivamente aprendió, con mejor Modelo encontrado
figure = plt.figure()
cols,rows = 2,3
i = 0 #subplot index
for row in range(1,rows+1):
    j = torch.randint(len(train_set),size=(1,)).item() # Los números aleatorios tambien se pueden generar desde pytorch. Util para trabajar en la GPU.
    #ploteamos la imagen original
    i += 1
    image,_ = train_set[j]
    figure.add_subplot(rows,cols,i)
    if row==1:
       plt.title("Original")
    plt.axis("off")
    plt.imshow(unbatch(image),cmap="Greys_r")
    #ploteamos la imagen predicha
    i += 1
    figure.add_subplot(rows,cols,i)
    plt.axis("off")
    if row ==1:
        plt.title("Predicha")
    image_pred = unbatch(model_conv2(batch(image)))
    plt.imshow(image_pred,cmap="Greys_r")
plt.show()
